In [ ]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from collections import deque
import matplotlib.pyplot as plt

In [ ]:
class A2C(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(A2C, self).__init__()
        self.fc1 = Dense(64, activation='relu')
        # self.fc2 = Dense(64, activation='relu')
        self.actor  = Dense(action_size, activation='softmax',
                                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        self.critic = Dense(1,
                                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        
    def call(self, x):
        x      = self.fc1(x)
        # x      = self.fc2(x)
        policy = self.actor(x)
        value  = self.critic(x)
        return policy, value

In [ ]:
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper params for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        
        self.model = A2C(self.state_size,self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)

        self.model.build(input_shape=(None, self.state_size))
        self.model.summary()
        
    def get_action(self, state):
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        policy, _ = self.model(state)
        policy = np.array(policy[0])
        return np.random.choice(self.action_size, 1, p=policy)[0]
        
    def train_model(self, state, action, reward, next_state, done):
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        next_state = tf.convert_to_tensor([next_state], dtype=tf.float32)
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            policy, value      = self.model(state)
            _,      next_value = self.model(next_state)
            target = reward + (1 - done) * self.discount_factor * next_value[0]
            
            # For policy network
            one_hot_action = tf.one_hot([action], self.action_size)
            action_prob = tf.reduce_sum(one_hot_action * policy, axis=1)
            cross_entropy = - tf.math.log(action_prob + 1e-5)
            advantage = tf.stop_gradient(target - value[0])
            actor_loss = tf.reduce_mean(cross_entropy * advantage)
            
            # For value network
            critic_loss = 0.5 * tf.square(tf.stop_gradient(target) - value[0])
            critic_loss = tf.reduce_mean(critic_loss)
            
            # integrate losses
            loss = 0.2 * actor_loss + critic_loss
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        return np.array(loss)

    def save_model(self):
        self.model.save_weights('./save_model/LunarLanderv2_a2c_TF', save_format='tf')

In [ ]:
%matplotlib tk

ENV_NAME = 'LunarLander-v2'
EPISODES = 5000
END_SCORE = 200

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = A2CAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores_avg, scores_raw, episodes, losses = [], [], [], []
    score_avg = 0
    
    end = False
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []
        
        state = env.reset()
        
        while not done:
            # env.render()

            # Interact with env.
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            loss = agent.train_model(state, action, reward, next_state, done)
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                # agent.update_target_model()

                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | loss: {:.4f}'.format(e, score_avg, np.mean(loss_list)))

                # Save data for plot
                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                losses.append(np.mean(loss_list))
                
                # View data
                if(e%10 == 0):
                    plt.subplot(211)
                    plt.plot(episodes, scores_avg, 'b')
                    plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                    plt.title('cartpole A2C')
                    plt.subplot(212)
                    plt.plot(episodes, losses, 'b')
                    plt.xlabel('episode'); plt.ylabel('loss'); plt.grid()
                    plt.savefig('./result/LunarLanderv2_a2c_TF.png')

                if score_avg > END_SCORE:
                    agent.save_model()
                    end = True
                    break
        if (end == True) | (e==EPISODES):
            np.save('./save_model/data/LunarLanderv2_a2c_TF_epi',episodes)
            np.save('./save_model/data/LunarLanderv2_a2c_TF_score',scores)
            np.save('./save_model/data/LunarLanderv2_a2c_TF_loss',losses)
            env.close()
            print("End")
            break